# Homework exercise 1
## Deadline: upload to Moodle by 19 November 18:00 h

__Suggestion: take this notebook and simply add your code and explanations. Your submission needs to include your code's output (even if the code throws an error).__

If you prefer to use .py files, you are expected to also include a PDF containing the output of your code and your explanations. Still, the code needs to be in a form that can be easily run on another computer.

If you use any file paths, assign paths to variables at the beginning of your code and use those variables when later referring to the paths.

__Name :__ Moritz-Jakob Leithner


The name of the file that you upload should be named *Homework1_YourLastName_YourStudentID*.

Reminder: you are required to attend class on 20 November to earn points for this homework exercise unless you have a valid reason for your absence.

You are encouraged to work on this exercise in teams of up to three students. If any part of the questions is unclear, please ask on the Moodle forum.

#### Simulating a stock market
Consider a financial market consisting of one or more stock exchanges. In this market, only one asset is traded (to keep it simple). 

Each stock exchange operates as a limit order book. I.e., if an investor Alice submits, e.g., a buy order, she specifies the maximum price she is willing to pay (called the limit price) and the number of shares she would like to trade (we will set the number of shares to 1 for simplicity). If there already is an existing sell order in the order book (previously submitted by another trader we will call Bob) with a limit price below or equal to Alice's limit price, a trade happens at Bob's limit price. If there is no such existing order, Alice's order enters the limit order book to wait for a seller who might be willing to trade at her limit price. Assume, for simplicity and contrary to real markets, that an order cannot be cancelled or modified after it has been submitted.

This problem set consists of tasks asking you to
* implement the functionality of a stock exchange operating as a limit order market
* generate random data representing orders submitted to the market
* simulate a market consisting of one or more stock exchanges using the randomly generated order flow
* finally interpret the output generated by your code

__You are expected to implement your solution using basic Python functionality plus numpy (in particular, for the generation of random numbers).__

1. Implementing a limit oder market

You are strongly encouraged to implement the limit order market as a class, so that you can later easily generate exchanges with slightly varying parameters.

The limit order market needs to offer the following functionality:

* access to the best bid and the best ask price (as attributes of the market or via methods)
* processing of incoming orders, where orders have the parameters `limit_price`, `side` (i.e. buy or sell), `size` (i.e. number of shares), and an `order_id` (i.e. an integer identifying an order). Processing constitutes of 
    * updating the order book (i.e. inserting the new order if it does not trade immediately, otherwise remove or modify the order against which the new order trades)
    * returning information on whether a trade happened and, if so, return trade price, size, and the IDs of the order that traded against one another
    
The only parameter pertaining to the order book is the tick size, which is a measure of the granularity of the prices the exchange allows. E.g., a tick size of 0.01 means that limit prices must be numbers expressed in whole cents.     

__Note:__ if there are multiple orders that would match with the new order, the buy order with the highest limit price/sell order with the lowest limit price will trade against the new order. If multiple orders at the same price exist in the order book, the order submitted the earliest get to trade first. This rule is called price-time priority and is the most common, though not the only, rule applied in real limit order books.

In [1]:
from datetime import datetime
import numpy as np
import pandas as pd
import bisect

class Order(object):
    number = 0

    def __init__(self,limit_price, timestamp = None):
        self.limit_price = limit_price
        self.timestamp = timestamp if timestamp is not None else datetime.now()
        self.ID = Order.number + 1
        Order.number += 1
        #print('Order (ID: %s) created' %(self.ID))
    
    def __str__(self):
        return " Limit Price: %s, Side: %s, Order ID: %s, Timestamp: %s" %(self.limit_price, self.side, self.ID, self.timestamp)

    def __del__(self):
        #print('Order (ID: %s) deleted' %(self.ID))
        pass

class Ask(Order):

        def __init__(self, limit_price, size = 1, side = 'sell', timestamp = None):
            timestamp = timestamp if timestamp is not None else datetime.now()

            Order.__init__(self, limit_price, timestamp)
            self.limit_price = limit_price
            self.size = size
            self.side = side


class Bid(Order):

        def __init__(self, limit_price, size = 1, side = 'buy', timestamp = None):
            timestamp = timestamp if timestamp is not None else datetime.now()

            Order.__init__(self, limit_price, timestamp)
            self.limit_price = limit_price
            self.size = size
            self.side = side


class OrderBook(object):
    def __init__(self, tick_size):
        self.tick_size = tick_size
        self.asks = []
        self.bids = []

        self.price_changes = []
        self.total_volume = 0
        self.spreads = []       #list of all spreads from Trades in OrderBook To calculate avg. spread
        self.hist_spreads = []  #list of the historical average spreads
        self.hist_std = []
        self.trade_prices = []


    def add_order(self, order):
        timestamp = datetime.now()   #when do we ever need this?
        order_key = (order.limit_price, order.timestamp)

        if order.side == 'sell':

            pos = bisect.bisect_right([(ask.limit_price, ask.timestamp) for ask in self.asks], order_key)
            self.asks.insert(pos, order)

            if not self.bids or order.limit_price <= self.get_best_bid():
                self.match_order(order, self.bids)
  
        elif order.side == 'buy':

            pos = bisect.bisect_left([(bid.limit_price, bid.timestamp) for bid in self.bids], order_key)
            self.bids.insert(pos, order)

            if not self.asks or order.limit_price >= self.get_best_ask():
                self.match_order(order, self.asks)

    def is_match(self, new_order, existing_order):
        if new_order.side == 'sell' and existing_order.side == 'buy':
            return new_order.limit_price <= existing_order.limit_price
        
        elif new_order.side == 'buy' and existing_order.side == 'sell':
            return new_order.limit_price >= existing_order.limit_price
        else:
            return False

    def match_order(self, new_order, orders):
        for index, order in enumerate(orders):
            if self.is_match(new_order, order):

                bid_price = new_order.limit_price if new_order.side == 'buy' else order.limit_price
                ask_price = new_order.limit_price if new_order.side == 'sell' else order.limit_price

                trade = Trade(new_order.ID if new_order.side == 'buy' else order.ID, order.ID if new_order.side =='buy' else new_order.ID, bid_price, ask_price, order.limit_price, 1)
                #print("Trade executed: ", trade)

                self.update_trade_metrics(trade)

                orders.pop(index)

                if new_order.side == 'sell':
                    self.asks = [ask for ask in self.asks if ask.ID != new_order.ID]
                else:
                    self.bids = [bid for bid in self.bids if bid.ID != new_order.ID]

                break
    
    def update_trade_metrics(self, trade):
        self.total_volume += trade.size
        self.spreads.append(trade.bid_price - trade.ask_price)

        average_spread = self.get_average_spread()
        self.hist_spreads.append(average_spread)

        if self.trade_prices:
            percentage_change = (trade.price / self.trade_prices[-1]) - 1
            self.price_changes.append(percentage_change)

        std_price_change = self.get_std_dp()
        self.hist_std.append(std_price_change)

        self.trade_prices.append(trade.price)
    
    def get_average_spread(self):
        if not self.spreads:
            return 0
        return sum(self.spreads) / len(self.spreads)
    
    def get_std_dp(self):
        if len(self.price_changes) < 2:
            return 0
        return np.std(self.price_changes)
    

    def get_best_ask(self):
        if not self.asks:
            return None
            
        best_ask = min(self.asks, key = lambda x: x.limit_price)
        return best_ask.limit_price

    def get_best_bid(self):
        if not self.bids:
            return None

        best_bid = max(self.bids, key = lambda x: x.limit_price)
        return best_bid.limit_price     

        
    def display(self):
        print("Total number of orders: %s" % Order.number)
        print("Max bid: %s" % self.get_best_bid())
        print("Min ask: %s" % self.get_best_ask())
        print("Bids: ")
        for bid in self.bids:
            print(bid)
        print("Asks: ")
        for ask in self.asks:
            print(ask)
        

class Trade(object):
    number_of_trades = 0

    def __init__(self, buy_order_id, sell_order_id, bid_price, ask_price, price, size):
        self.buy_order_id = buy_order_id
        self.sell_order_id = sell_order_id
        self.bid_price = bid_price
        self.ask_price = ask_price
        self.price = price
        self.size = size
        Trade.number_of_trades += 1

    def __str__(self):
        return "(%s) : Buy Order ID: %s, Sell Order ID: %s, Price: %s/%s -> %s, Size: %s" % (Trade.number_of_trades, self.buy_order_id, self.sell_order_id, self.bid_price, self.ask_price, self.price, self.size)


2. Generating random numbers and simulating order flow

You are asked to allow the orders submitted to the market to depend on the following parameters:

* the fair value of the security
* a private value an individual trader experiences from holding the asset. This private value reflect how much the trader likes or dislikes owning the asset

We are going to assume the following simple rule determining side and limit price: if the private value is positive (negative), the trader will submit a buy (sell) order at a price equal to the sum of fair value and private value minus (plus) a constant `k`, which we will interpret as the required "profit", rounded down (up) to the nearest tick. This rule is not intended to reflect optimal behavior by the traders but rather serves to get a relatively reasonable simulation of the trading process. 

Write a function that generates the orders based on the parameters described above. Generate 100,000 random values following a standard normal distribution. Also

* generate 100,000 random numbers following the uniform distribution between 0 and 1, which we will use to determine changes to the fair value in the next step
* generate 100,000 binary random numbers with equal probability, which we will use to determine whether any change to the fair value is positive or negative
* generate 100,000 binary random numbers with equal probability, which we will use to determine the exchange to which an order is submitted if the trader is indifferent in the setting with two exchanges

In [2]:
def mk_order(fv, pv, k, exchange1, exchange2):
    '''individual orders based on params:
    - fv: fair value
    - pv: private value
    - k: profit constant
    - exchange1: the first exchange order book
    - exchange2: the second exchange order book
    '''

    limit_price = ((fv + pv - k) / exchange1.tick_size)//1 * exchange1.tick_size if pv > 0 else ((fv + pv + k) / exchange2.tick_size)//1 * exchange2.tick_size
    order_side = 'buy' if pv > 0 else 'sell'
    order = Bid(limit_price) if order_side == 'buy' else Ask(limit_price)

    chosen_exchange = None

    if order_side == 'sell':
        best_bid_e1 = exchange1.get_best_bid()
        best_bid_e2 = exchange2.get_best_bid()

        if best_bid_e1 is not None and best_bid_e2 is not None:
            chosen_exchange = exchange1 if best_bid_e1 > best_bid_e2 else exchange2
        elif best_bid_e1 is not None and limit_price <= best_bid_e1:
            chosen_exchange = exchange1
        elif best_bid_e2 is not None and limit_price <= best_bid_e2:
            chosen_exchange = exchange2
    else:  #BUY
        best_ask_e1 = exchange1.get_best_ask()
        best_ask_e2 = exchange2.get_best_ask()
        
        if best_ask_e1 is not None and best_ask_e2 is not None:
            chosen_exchange = exchange1 if best_ask_e1 < best_ask_e2 else exchange2
        elif best_ask_e1 is not None and limit_price >= best_ask_e1:
            chosen_exchange = exchange1
        elif best_ask_e2 is not None and limit_price >= best_ask_e2:
            chosen_exchange = exchange2

    if chosen_exchange is None:
        chosen_exchange = rng.choice([exchange1, exchange2])

    chosen_exchange.add_order(order)

    return order, chosen_exchange



#Set up for simulations
tick = [0.01, 0.1, 1]
Q = [0,0.001, 0.01, 0.1]
K = [0.1, 1]
fair_value = 100


rng = np.random.default_rng(1234)

A = rng.standard_normal(10000)
A1 = 0.1 * A
PrV = [A, A1]


B = rng.uniform(0, 1, 10000)
C = rng.choice([-1,1], 10000)


3. Simulating the market

Based on what you implemented in the previous steps, please simulate a market for all combinations of the following parameters:

* Initital fair value: 100
* Constant size for all orders: 1
* Probability of an increase/decrease of the fair value by 1 in each period (1 order = 1 period): 0, 0.001, 0.01, 0.1
* Parameter `k` (see above): 0.1, 1
* Standard deviation of private values: 0.1, 1
* Exchanges: 
    * one exchange with tick size 0.01, 0.1, 1
    * two exchanges each with tick size 0.01, 0.1, 1
    * one exchange with tick size 0.01, another with tick size 0.1 
    * one exchange with tick size 0.1, another with tick size 1 

Rules for order submission with multiple exchanges:

* If the order can immediately trade on one of the two exchanges, send the order to that exchange
* If the order can immediately trade on neither of the exchanges, send the order with equal probability to either of the exchanges
* If the order can immediately trade on both exchanges, send the order to the exchange offering the better price

For each simulation, compute the total trading volume, the average bid-ask spread (simple difference between ask and bid) on each exchange, and the standard deviation of changes in trade prices. For simulations containing two exchanges, also compute the trading volumes on each exchange.

In [3]:
simulation1 = []  

for A in PrV:
    for k in K:
        for q in Q:
            
            exchange1 = OrderBook(tick[0])
            fair_value = 100

            for pv, prchange, dirchange in zip(A, B, C):
                if prchange < q:
                    fair_value += dirchange
                else:
                    fair_value += dirchange * 0
                order = mk_order(fair_value, pv, k, exchange1, exchange1)

            simulation1.append({
                'k': k,
                'q': q,
                'total_vol': exchange1.total_volume,
                'tick1': exchange1.tick_size,
                'avg_spread': exchange1.get_average_spread(),
                'sigma_price_change1': exchange1.get_std_dp()
            })


4. Try to interpret the results computed above. How do trading volume, price volatility, and (for the setting with two exchanges) the proportion of trades on each exchange depend on the parameters of the simulation?

In [4]:
df1 = pd.DataFrame(simulation1)
print(df1)

      k      q  total_vol  tick1  avg_spread  sigma_price_change1
0   0.1  0.000       4978   0.01    1.405408             0.007248
1   0.1  0.001       4976   0.01    1.397657             0.007477
2   0.1  0.010       4978   0.01    1.383373             0.007944
3   0.1  0.100       4973   0.01    1.606443             0.011323
4   1.0  0.000       3127   0.01    0.169850             0.006437
5   1.0  0.001       3826   0.01    0.174537             0.007702
6   1.0  0.010       4401   0.01    0.686162             0.016626
7   1.0  0.100       4622   0.01    1.513711             0.010996
8   0.1  0.000       3158   0.01    0.016501             0.000634
9   0.1  0.001       4009   0.01    0.365318             0.006160
10  0.1  0.010       4521   0.01    0.856235             0.016038
11  0.1  0.100       4658   0.01    1.666668             0.010418
12  1.0  0.000          0   0.01    0.000000             0.000000
13  1.0  0.001        891   0.01    0.098114             0.000338
14  1.0  0

### 1 Exchange, tick = 0.01

16 simulations: [0]-[7]: the private values are Standard Normal (A), and for the others the private value is Gaussian with zero mean and std dev of 0.1 (B)

* #### A:
 Trading volume is fairly constant over all params. The average spread is significantly higher for the smaller profit constant, except for the constellation with high k and high chance of change in fair price, q. The change in volatility does not seem to be following a specific direction dependent on a parameter change

* #### B:
Trading volume changes significantly, and without a change in the fair price it is in one instance not possible to find matching orders at all. The smaller changes in private value are not able to span the distance needed for the higher profit requirements of the agents. The price volatility increases symmetrically with the increase in fair value volatility, so does the total trading volume. This effect is larger for the higher profit requirement, which might also be due to the small sample size and just show a general increase in trades and prices. UNfortunately a simulation with 100k orders what not feasible to compile in less than 30' with this code. 


In [5]:
simulation2 = []  # Define the simulation_results list

for A in PrV:
    for k in K:
        for q in Q:
            
            exchange1 = OrderBook(tick[1])
            fair_value = 100

            for pv, prchange, dirchange in zip(A, B, C):
                if prchange < q:
                    fair_value += dirchange
                else:
                    fair_value += dirchange * 0
                order = mk_order(fair_value, pv, k, exchange1, exchange1)

            simulation2.append({
                'k': k,
                'q': q,
                'total_vol': exchange1.total_volume,
                'tick1': exchange1.tick_size,
                'avg_spread': exchange1.get_average_spread(),
                'sigma_price_change1': exchange1.get_std_dp()
            })


In [6]:
df2 = pd.DataFrame(simulation2)
print(df2)

      k      q  total_vol  tick1  avg_spread  sigma_price_change1
0   0.1  0.000       4978    0.1    1.407292             0.007277
1   0.1  0.001       4976    0.1    1.399759             0.007470
2   0.1  0.010       4978    0.1    1.384411             0.007979
3   0.1  0.100       4973    0.1    1.606395             0.011396
4   1.0  0.000       3158    0.1    0.165009             0.006358
5   1.0  0.001       3856    0.1    0.166390             0.007648
6   1.0  0.010       4423    0.1    0.658942             0.016810
7   1.0  0.100       4634    0.1    1.457186             0.010868
8   0.1  0.000       3205    0.1    0.014727             0.000709
9   0.1  0.001       4052    0.1    0.353825             0.005745
10  0.1  0.010       4571    0.1    0.776745             0.014382
11  0.1  0.100       4710    0.1    1.422251             0.010558
12  1.0  0.000          0    0.1    0.000000             0.000000
13  1.0  0.001        891    0.1    0.135690             0.000338
14  1.0  0

### 1 Exchange, tick = 0.1

16 simulations: [0]-[7]: the private values are Standard Normal (A), and for the others the private value is Gaussian with zero mean and std dev of 0.1 (B)

* #### A:
 Similarly with the 0.01 tick, the trading volume is relatively constant and in most cases almost all orders are traded. Also similarly to the other tick size: the smaller profit requirement for the individuals leads to larger bid-ask spreads, except for the asset with the most volatile fair value. The price volatility is generally developing less salient for lower k when the fair value volatility is increased.

* #### B:
Trading volume changes is relatively constant for small k, but changes significantly for large k.  Without a change in the fair price it is in one instance, again, not possible to find matching orders at all. While, for small k, an increase in fair value volatility is correlated to an increase in trading volume, the accompanied increase in price volatility is like in the first simulation. the average spread also increases significantly (ca 100x between low volatility and highest vol for the fair price).


In [7]:
simulation3 = []  

for A in PrV:
    for k in K:
        for q in Q:
            
            exchange1 = OrderBook(tick[2])
            fair_value = 100

            for pv, prchange, dirchange in zip(A, B, C):
                if prchange < q:
                    fair_value += dirchange
                else:
                    fair_value += dirchange * 0
                order = mk_order(fair_value, pv, k, exchange1, exchange1)

            simulation3.append({
                'k': k,
                'q': q,
                'total_vol': exchange1.total_volume,
                'tick1': exchange1.tick_size,
                'avg_spread': exchange1.get_average_spread(),
                'sigma_price_change1': exchange1.get_std_dp()
            })


In [8]:
df3 = pd.DataFrame(simulation3)
print(df3)

      k      q  total_vol  tick1  avg_spread  sigma_price_change1
0   0.1  0.000       4978      1    1.461832             0.007904
1   0.1  0.001       4978      1    1.448775             0.008180
2   0.1  0.010       4978      1    1.434914             0.008654
3   0.1  0.100       4975      1    1.640201             0.011451
4   1.0  0.000       3205      1    0.147270             0.007127
5   1.0  0.001       4095      1    0.212454             0.007928
6   1.0  0.010       4614      1    0.497399             0.015218
7   1.0  0.100       4737      1    1.063120             0.011497
8   0.1  0.000       3205      1    0.000000             0.007118
9   0.1  0.001       4052      1    0.117966             0.007317
10  0.1  0.010       4572      1    0.471129             0.014352
11  0.1  0.100       4713      1    1.093571             0.010739
12  1.0  0.000          0      1    0.000000             0.000000
13  1.0  0.001       2445      1    0.285072             0.000461
14  1.0  0

### 1 Exchange, tick = 1

16 simulations: [0]-[7]: the private values are Standard Normal (A), and for the others the private value is Gaussian with zero mean and std dev of 0.1 (B)

Trading volume is very large everywhere except for our non-trading market. Probably because the high tick-size levels the nuances of supply and demand.

Price Volatility and Average Bid-Ask spread behave in the usual way to the first two simulations. They are even somewhat similar in absolute terms, which I find surprising just from intuitively expecting larger spreads because of larger tick sizes.

In [9]:
simulation4 = []  # Define the simulation_results list

for A in PrV:
    for k in K:
        for q in Q:
            
            exchange1 = OrderBook(tick[0])
            exchange2 = OrderBook(tick[0])
            fair_value = 100

            for pv, prchange, dirchange in zip(A, B, C):
                if prchange < q:
                    fair_value += dirchange
                else:
                    fair_value += dirchange * 0
                order = mk_order(fair_value, pv, k, exchange1, exchange2)


            simulation4.append({
                'k': k,
                'q': q,
                'total_vol': exchange1.total_volume + exchange2.total_volume,
                'vol1': exchange1.total_volume,
                'vol2': exchange2.total_volume,
                'tick1': exchange1.tick_size,
                'tick2':exchange2.tick_size,
                'avg_spread1': exchange1.get_average_spread(),
                'avg_spread2': exchange2.get_average_spread(),
                'sigma_price_change1': exchange1.get_std_dp(),
                'sigma_price_change2': exchange2.get_std_dp()
            })


In [10]:
df4 = pd.DataFrame(simulation4)
print(df4)

      k      q  total_vol  vol1  vol2  tick1  tick2  avg_spread1  avg_spread2  \
0   0.1  0.000       4978  2486  2492   0.01   0.01     1.413910     1.396926   
1   0.1  0.001       4976  2483  2493   0.01   0.01     1.417374     1.378018   
2   0.1  0.010       4978  1690  3288   0.01   0.01     1.435834     1.366217   
3   0.1  0.100       4951  1595  3356   0.01   0.01     1.843248     1.740861   
4   1.0  0.000       3127  3119     8   0.01   0.01     0.169769     0.243750   
5   1.0  0.001       3824     7  3817   0.01   0.01     0.472857     0.174467   
6   1.0  0.010       4401  4316    85   0.01   0.01     0.679307     1.178000   
7   1.0  0.100       4612  4349   263   0.01   0.01     1.529352     1.571179   
8   0.1  0.000       3158     7  3151   0.01   0.01     0.021429     0.016515   
9   0.1  0.001       3999   116  3883   0.01   0.01     0.395259     0.367937   
10  0.1  0.010       4448  1017  3431   0.01   0.01     0.783972     0.995068   
11  0.1  0.100       4609  2

### 2 Exchange, ticks = 0.01

16 simulations: [0]-[7]: the private values are Standard Normal (A), and for the others the private value is Gaussian with zero mean and std dev of 0.1 (B)

* #### A:
 Total trading volume seems generally quite high. Although we can see that there are instances where it is spread out evenly between the exchanges and other instances where almost all trading happens in one of the two. This happens throughout the parameter configuration, I wouldn't be able to comment confidently on any connection with the individual trading volumes other than momentum.

* #### B:
here there's no trade at all in one, the usual, instance. And another where there is only trade at one of the two exchanges. This seems en par with the suggestions we have seen for the single exchange case, where the stationary value of the asset and the high profit requirements of the agents lead to a non-trade outcome.

In [11]:
simulation5 = []  

for A in PrV:
    for k in K:
        for q in Q:
            
            exchange1 = OrderBook(tick[1])
            exchange2 = OrderBook(tick[1])
            fair_value = 100

            for pv, prchange, dirchange in zip(A, B, C):
                if prchange < q:
                    fair_value += dirchange
                else:
                    fair_value += dirchange * 0
                order = mk_order(fair_value, pv, k, exchange1, exchange2)

            total_vol = exchange1.total_volume + exchange2.total_volume

            simulation5.append({
                'k': k,
                'q': q,
                'total_vol': exchange1.total_volume + exchange2.total_volume,
                'vol1': exchange1.total_volume,
                'vol2': exchange2.total_volume,
                'tick1': exchange1.tick_size,
                'tick2':exchange2.tick_size,
                'avg_spread1': exchange1.get_average_spread(),
                'avg_spread2': exchange2.get_average_spread(),
                'sigma_price_change1': exchange1.get_std_dp(),
                'sigma_price_change2': exchange2.get_std_dp()
            })


In [12]:
df5 = pd.DataFrame(simulation5)
print(df5)

      k      q  total_vol  vol1  vol2  tick1  tick2  avg_spread1  avg_spread2  \
0   0.1  0.000       4978  2393  2585    0.1    0.1     1.405725     1.408743   
1   0.1  0.001       4976  2470  2506    0.1    0.1     1.403482     1.396089   
2   0.1  0.010       4978  2509  2469    0.1    0.1     1.372619     1.410409   
3   0.1  0.100       4954  2374  2580    0.1    0.1     1.602780     1.886744   
4   1.0  0.000       3157     7  3150    0.1    0.1     0.185714     0.165429   
5   1.0  0.001       3850  1539  2311    0.1    0.1     0.143730     0.197750   
6   1.0  0.010       4422   218  4204    0.1    0.1     1.029358     0.642269   
7   1.0  0.100       4610   559  4051    0.1    0.1     1.214132     1.572007   
8   0.1  0.000       3204     5  3199    0.1    0.1     0.040000     0.014723   
9   0.1  0.001       4050    22  4028    0.1    0.1     0.304545     0.354767   
10  0.1  0.010       4508   108  4400    0.1    0.1     5.435185     0.724818   
11  0.1  0.100       4666  2

### 2 Exchanges, tick = 0.1

16 simulations: [0]-[7]: the private values are Standard Normal (A), and for the others the private value is Gaussian with zero mean and std dev of 0.1 (B)

The results are very similar to the ones before. Tick size seems to not have that much of an influence on the volume, spread, and volatility.

In [13]:
simulation6 = [] 

for A in PrV:
    for k in K:
        for q in Q:
            
            exchange1 = OrderBook(tick[2])
            exchange2 = OrderBook(tick[2])
            fair_value = 100

            for pv, prchange, dirchange in zip(A, B, C):
                if prchange < q:
                    fair_value += dirchange
                else:
                    fair_value += dirchange * 0
                order = mk_order(fair_value, pv, k, exchange1, exchange2)

            total_vol = exchange1.total_volume + exchange2.total_volume

            simulation6.append({
                'k': k,
                'q': q,
                'total_vol': exchange1.total_volume + exchange2.total_volume,
                'vol1': exchange1.total_volume,
                'vol2': exchange2.total_volume,
                'tick1': exchange1.tick_size,
                'tick2':exchange2.tick_size,
                'avg_spread1': exchange1.get_average_spread(),
                'avg_spread2': exchange2.get_average_spread(),
                'sigma_price_change1': exchange1.get_std_dp(),
                'sigma_price_change2': exchange2.get_std_dp()
            })


In [14]:
df6 = pd.DataFrame(simulation6)
print(df6)

      k      q  total_vol  vol1  vol2  tick1  tick2  avg_spread1  avg_spread2  \
0   0.1  0.000       4978  2532  2446      1      1     1.491706     1.430908   
1   0.1  0.001       4978  2474  2504      1      1     1.454729     1.442891   
2   0.1  0.010       4978  2075  2903      1      1     1.426506     1.447813   
3   0.1  0.100       4954  1616  3338      1      1     1.981436     1.702816   
4   1.0  0.000       3205     7  3198      1      1     0.142857     0.147280   
5   1.0  0.001       4093    20  4073      1      1     0.200000     0.213111   
6   1.0  0.010       4544   145  4399      1      1     4.144828     0.456695   
7   1.0  0.100       4685  1324  3361      1      1     0.638973     1.508777   
8   0.1  0.000       3205     6  3199      1      1     0.000000     0.000000   
9   0.1  0.001       4049    26  4023      1      1     0.115385     0.118817   
10  0.1  0.010       4510   106  4404      1      1     4.452830     0.440736   
11  0.1  0.100       4696   

### 2 Exchanges, tick = 1

16 simulations: [0]-[7]: the private values are Standard Normal (A), and for the others the private value is Gaussian with zero mean and std dev of 0.1 (B)

Slightly higher trading volumes, especially with the problematic cases of low change in private valuations and high profit requirements.

In [15]:
simulation7 = []  

for A in PrV:
    for k in K:
        for q in Q:
            
            exchange1 = OrderBook(tick[0])
            exchange2 = OrderBook(tick[1])
            fair_value = 100

            for pv, prchange, dirchange in zip(A, B, C):
                if prchange < q:
                    fair_value += dirchange
                else:
                    fair_value += dirchange * 0
                order = mk_order(fair_value, pv, k, exchange1, exchange2)

            total_vol = exchange1.total_volume + exchange2.total_volume

            simulation7.append({
                'k': k,
                'q': q,
                'total_vol': exchange1.total_volume + exchange2.total_volume,
                'vol1': exchange1.total_volume,
                'vol2': exchange2.total_volume,
                'tick1': exchange1.tick_size,
                'tick2':exchange2.tick_size,
                'avg_spread1': exchange1.get_average_spread(),
                'avg_spread2': exchange2.get_average_spread(),
                'sigma_price_change1': exchange1.get_std_dp(),
                'sigma_price_change2': exchange2.get_std_dp()
            })


In [16]:
df7 = pd.DataFrame(simulation7)
print(df7)

      k      q  total_vol  vol1  vol2  tick1  tick2  avg_spread1  avg_spread2  \
0   0.1  0.000       4978  2430  2548   0.01    0.1     1.460082     1.442712   
1   0.1  0.001       4976  2497  2479   0.01    0.1     1.440236     1.447075   
2   0.1  0.010       4978  2518  2460   0.01    0.1     1.418431     1.466919   
3   0.1  0.100       4951  1090  3861   0.01    0.1     1.988514     1.774307   
4   1.0  0.000       3152    10  3142   0.01    0.1     0.368000     0.200722   
5   1.0  0.001       3840  3815    25   0.01    0.1     0.208708     0.296800   
6   1.0  0.010       4418    88  4330   0.01    0.1     1.528523     0.702704   
7   1.0  0.100       4559  1018  3541   0.01    0.1     0.753910     1.993287   
8   0.1  0.000       3205     7  3198   0.01    0.1     0.037143     0.039400   
9   0.1  0.001       4049    23  4026   0.01    0.1     0.265217     0.386900   
10  0.1  0.010       4561   275  4286   0.01    0.1     1.261018     0.794701   
11  0.1  0.100       4649  2

### 2 Exchanges, ticks = [0.1, 0.01]

16 simulations: [0]-[7]: the private values are Standard Normal (A), and for the others the private value is Gaussian with zero mean and std dev of 0.1 (B)

Surprisingly there are more low volume cases for the case with the smaller tick, again, my intuition here would've taken my into the completely other direction. Independent of their individual volume, the two exchanges report very similar results for average spreads and price volatility.

In [17]:
simulation8 = []  

for A in PrV:
    for k in K:
        for q in Q:
            
            exchange1 = OrderBook(tick[2])
            exchange2 = OrderBook(tick[1])
            fair_value = 100

            for pv, prchange, dirchange in zip(A, B, C):
                if prchange < q:
                    fair_value += dirchange
                else:
                    fair_value += dirchange * 0
                order = mk_order(fair_value, pv, k, exchange1, exchange2)

            total_vol = exchange1.total_volume + exchange2.total_volume

            simulation8.append({
                'k': k,
                'q': q,
                'total_vol': exchange1.total_volume + exchange2.total_volume,
                'vol1': exchange1.total_volume,
                'vol2': exchange2.total_volume,
                'tick1': exchange1.tick_size,
                'tick2':exchange2.tick_size,
                'avg_spread1': exchange1.get_average_spread(),
                'avg_spread2': exchange2.get_average_spread(),
                'sigma_price_change1': exchange1.get_std_dp(),
                'sigma_price_change2': exchange2.get_std_dp()
            })


In [18]:
df8 = pd.DataFrame(simulation8)
print(df8)

      k      q  total_vol  vol1  vol2  tick1  tick2  avg_spread1  avg_spread2  \
0   0.1  0.000       4976  2403  2573      1    0.1     0.990470     0.978857   
1   0.1  0.001       4975  2461  2514      1    0.1     0.976189     0.978202   
2   0.1  0.010       4976  2609  2367      1    0.1     1.019241     0.974989   
3   0.1  0.100       4904  2183  2721      1    0.1     1.611681     1.479897   
4   1.0  0.000       1893    20  1873      1    0.1     0.470000     0.454138   
5   1.0  0.001       2512     4  2508      1    0.1     0.675000     0.466906   
6   1.0  0.010       3749   265  3484      1    0.1     2.621132     1.082377   
7   1.0  0.100       4372   345  4027      1    0.1     1.676522     2.318997   
8   0.1  0.000       1609     2  1607      1    0.1     0.100000     0.113255   
9   0.1  0.001       3244   422  2822      1    0.1     0.036493     0.354571   
10  0.1  0.010       4175   953  3222      1    0.1     1.230010     0.807356   
11  0.1  0.100       4496  2

### 2 Exchanges, ticks = [0.1, 1]

16 simulations: [0]-[7]: the private values are Standard Normal (A), and for the others the private value is Gaussian with zero mean and std dev of 0.1 (B)

There's suspiciously low trading volume for (A), high k, which is untypical when compared to the other cases. Now there are some cases where the average spread seem to differ vastly between the two exchanges. 